# A script to plot and fit curves to a given functional shape

In [2]:
## Most of the plot shapes and packages taken from https://plotly.com/python/v3/peak-fitting/
## Mixed with data fitting from https://github.com/emilyripka/BlogRepo/blob/master/181119_PeakFitting.ipynb

# All these packages are to us plotly. plotly allows creation of interactive plots, 
#in the notebook or in an html file. 
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio
###################################################################################

import numpy as np # Tools for working with floating point numbers among other things
import pandas as pd # Tools for reading and processing data
import scipy
import peakutils # Tools for peak finding and fitting

from scipy import signal

# Basic plotting concepts that will be used below

In [3]:
#-- opens a browser tab for visualization when needed. Comment if you want notebook visualization.--
pio.renderers.default = 'browser' 

#--simple figure creation--
fig = go.Figure(go.Scatter(x=[1, 2, 3, 4], y=[4, 3, 2, 1])) 
fig.update_layout(title_text='hello world') 

""" pio.write_html can save the plot as an html file.
We will set auto_open=False in subsequent calls, since updating the html file 
in the browser reloads the figure in the browser tab automatically."""
pio.write_html(fig, file='plot.html', auto_open=True) 

# --This line is for showing the figure in the notebook.--
#pio.show(fig) 

# Reading the data, and taking a look at it

In [5]:
#--Reads data from a csv file--
ldos = pd.read_csv('./LDOS.dat')
df = ldos

#--creates a table to plot--
table = ff.create_table(df,height_constant=60)

list(ldos.columns)
print(ldos['LDOS'][0:10])

#--to show the table--
#table.show()

0    0.071005
1    0.071126
2    0.071246
3    0.071367
4    0.071488
5    0.071610
6    0.071732
7    0.071854
8    0.071977
9    0.072100
Name: LDOS, dtype: float64


# Section to highlight a section of the plot

In [15]:
#--Define the left and right end points of the region of interest--
#--It may be a good idea to plot the full range and check the interactive plot to get the region of interest.
left_endpt=700
right_endpt=900 #len(ldos['LDOS'])

#--To plot, we first construct our curves, that are called traces here--
original_trace = go.Scatter(
    x = [j for j in range(len(ldos['LDOS']))],
    y = ldos['LDOS'][0:left_endpt].tolist() + [None for k in range(right_endpt - left_endpt)] +
        ldos['LDOS'][right_endpt + 1:len(ldos['LDOS'])].tolist(),
    mode = 'lines',
    name = 'LDOS full range',
    marker = dict(color = 'rgb(160,200,250)')
)

highlighted_trace = go.Scatter(
    x = [j for j in range(left_endpt, right_endpt)],
    y = ldos['LDOS'][left_endpt:right_endpt],
    mode = 'lines',
    name = 'Highlighted Section',
    marker = dict(color = 'rgb(0,56,210)')
)

#--Here we join the two curves into one array--
ldos_data= [original_trace, highlighted_trace,]

#--The go.Figure takes this array to plot--
fig=go.Figure(ldos_data)

#--Sets the title--
fig.update_layout(title_text='LDOS')

#--Writes the html file of the plot--
pio.write_html(fig, file='plot.html', auto_open=False)
#pio.show(fig, filename='stock-stock_data-LDOS')

# Section to find the peaks and mark them in the plot

In [18]:
#--First we define the x and y array of the region of interest.
x = [j for j in range(len(ldos))][left_endpt:right_endpt]
y = ldos['LDOS'][left_endpt:right_endpt]

#-- y is transformed into a list.
y = y.tolist()


#--And then into a numerical array.
cb = np.array(y)

"""peakutils.indexes gives the indices of the peaks found in the list y, 
within some threshold (difference of peak and surrounding values) 
and with a minimum distance between peaks"""
indices = peakutils.indexes(cb, thres=0.25, min_dist=0.1)

trace = go.Scatter(
    x=x,
    y=y,
    mode='lines',
    marker=dict(
        color='rgb(0,56,210)'
    ),
    name='Highlighted Plot'
)

trace2 = go.Scatter(
    x=indices + left_endpt,
    y=[y[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='rgb(255,0,0)',
        symbol='cross'
    ),
    name='Detected Peaks'
)

data = [trace, trace2]
fig=go.Figure(data)
fig.update_layout(title_text='LDOS peaks')
pio.write_html(fig, file='plot.html', auto_open=False)
#pio.show(fig)
#py.iplot(stock_data, filename='stock-stock_data-with-peaks')

# Here we define our functions to fit and fit them to the data

In [21]:
#def gaussian(x, mu, sig):
#    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

def _1gaussian(x, amp1,cen1,sigma1):
    return amp1*(1/(sigma1*(np.sqrt(2*np.pi))))*(np.exp((-1.0/2.0)*(((x-cen1)/sigma1)**2)))

def _1lorentzian(x, amp, cen, wid):
    return amp*wid**2/((x-cen)**2+wid**2)



#first_index = indices[4]
#--The bound to the fitting data. Could be smaller than the region of interest.
left_gauss_bound = left_endpt
right_gauss_bound = right_endpt

#--Define x and y arrays to be used in the fitting.
x_values_1 = np.asarray(x[left_gauss_bound-left_endpt:right_gauss_bound-left_endpt])
y_values_1 = np.asarray(y[left_gauss_bound-left_endpt:right_gauss_bound-left_endpt])

"""scipy.optimize.curve_fit ,  With method='lm', the algorithm uses the Levenberg-Marquardt algorithm through leastsq. 
Note that this algorithm can only deal with unconstrained problems. 
It needs at least as many points as parameters to fit"""

gaussian_initial_guesses=[800.0,840,40]
lorentzian_initial_guesses=[20.0,840,40]

#--For better fitting in the Gaussian we substract from the y values the first point in the range.
popt_gauss, pcov_gauss = scipy.optimize.curve_fit(_1gaussian,
                                                  x_values_1, 
                                                  y_values_1-y_values_1[0], 
                                                  p0=gaussian_initial_guesses)
#--Error parameters
perr_gauss = np.sqrt(np.diag(pcov_gauss))

popt_lorentz, pcov_lorentz = scipy.optimize.curve_fit(_1lorentzian, 
                                                      x_values_1, 
                                                      y_values_1, 
                                                      p0=lorentzian_initial_guesses)
perr_lorentz = np.sqrt(np.diag(pcov_lorentz))


#--Linear space to evaluate the fitted functions.
xvals=np.linspace(x_values_1[0],x_values_1[-1],2000)
gaussian_y_1 = [_1gaussian(x_dummy, *popt_gauss) for x_dummy in xvals]
lorentzian_y_1 = [_1lorentzian(x_dummy, *popt_lorentz) for x_dummy in xvals]

#--This cell prints the fitting parameters with their errors
print("Gaussian fit")
print("amplitude = %0.2f (+/-) %0.2f" % (popt_gauss[0], perr_gauss[0]))
print( "center = %0.2f (+/-) %0.2f" % (popt_gauss[1], perr_gauss[1]))
print( "sigma = %0.2f (+/-) %0.2f" % (popt_gauss[2], perr_gauss[2]))


#--This cell prints the fitting parameters with their errors
print("Lorentzian fit")
print("amplitude = %0.2f (+/-) %0.2f" % (popt_lorentz[0], perr_lorentz[0]))
print( "center = %0.2f (+/-) %0.2f" % (popt_lorentz[1], perr_lorentz[1]))
print( "sigma = %0.2f (+/-) %0.2f" % (popt_lorentz[2], perr_lorentz[2]))


trace = go.Scatter(
    x=x,
    y=y,
    mode='lines',
    marker=dict(
        color='rgb(0,56,210)'
    ),
    name='Highlighted Plot'
)

trace2 = go.Scatter(
    x=indices + left_endpt,
    y=[y[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='rgb(255,0,0)',
        symbol='cross'
    ),
    name='Detected Peaks'
)

trace3 = go.Scatter(
    #x=x_values_1,
    x=[item_x for item_x in xvals],
    y=[item_y + y_values_1[0] for item_y in gaussian_y_1],
    mode='lines',
    marker=dict(
        size=2,
        color='rgb(200,0,250)',
    ),
    name='Gaussian Fit'
)

trace4 = go.Scatter(
    #x=x_values_1,
    x=[item_x for item_x in xvals],
    y=[item_y for item_y in lorentzian_y_1 ],
    mode='lines',
    marker=dict(
        size=2,
        color='rgb(0,256,256)',
    ),
    name='Lorentzian Fit'
)

data = [trace, trace2,trace3, trace4]
fig=go.Figure(data)
fig.update_layout(title_text='LDOS with peaks and fit')
pio.write_html(fig, file='plot.html', auto_open=False)
#pio.show(fig)
#py.iplot(data, filename='stock-data-with-peaks-and-fit')

Gaussian fit
amplitude = 791.61 (+/-) 4.53
center = 842.89 (+/-) 0.26
sigma = 40.69 (+/-) 0.29
Lorentzian fit
amplitude = 9.16 (+/-) 0.01
center = 843.44 (+/-) 0.05
sigma = 49.92 (+/-) 0.09
